# Ex ? Manipulating Spectra Part 2 - `Spectra`

## Notebook Initialization

In [25]:
%load_ext autoreload
import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
%matplotlib inline

In [27]:
%autoreload


import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

import numpy as np
from scipy.integrate import quad

import darkhistory.physics as phys
import darkhistory.spec.spectools as spectools
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra

In [4]:
eng = np.array([1, 10, 100, 1000])
spec_arr = [Spectrum(eng, np.ones(4)*i, rs=100, spec_type='N') for i in np.arange(4)]
test_spectra = Spectra(spec_arr)
test_spectra.redshift(np.array([0.01, 0.1, 1, 10]))
print(test_spectra.grid_vals)

[[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [2. 2. 0. 0.]
 [3. 3. 3. 0.]]


In [5]:
a = Spectrum(eng, np.ones(4)*3, rs=100, spec_type='N') 
a.totN(bound_type='bin')

12.0

In [9]:
0.5*13*10 + 14*100 + 15*1000

16465.0

In [11]:
9*0.5 + 10 + 11

25.5

In [16]:
12*1 + 13*10 + 14*100 + 15*1000

16542

In [17]:
1*10 + 2*100

210

In [36]:
%autoreload 2
phys.Tm_std(100)

array(0.01421714)

In [37]:
a = phys.struct_boost_func()

In [41]:
a(1)

17090553.829749867